In [ ]:
import numpy as np
from qutip import basis, ket2dm, tensor, expect, fidelity

# Define the states
H = basis(2,0)
V = basis(2,1)

## Maximally entangled EPR pairs:
phi_plus = (tensor(H, H) + tensor(V, V)).unit()

## Maximally entangled EPR pairs with mixture:
rho = 0.7*ket2dm(phi_plus) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

## Non-maximally entangled pair:
phi = (1/np.sqrt(5)*tensor(H,H) + 2/np.sqrt(5)*tensor(V,V)).unit()


## Non-maximally entangled pair with mixture:
rho_prime = 0.7*ket2dm(phi) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

### Simulation of photon generation

In [2]:
def simulate_detection(rate, eff, dark_rate, dead_time, totol_time):
    t, counts = 0, 0
    dead = False
    while t < totol_time:
        if dead:
            t+= dead_time
            dead = False
        else:  
            photon_interval = np.random.exponential(1/rate)
            dark_interval = np.random.exponential(1/dark_rate)
            dt = min(photon_interval, dark_interval)
            t += dt
            if np.random.rand() < eff:
                counts += 1
                dead = True
    return counts

### Basis selection and Bell measurement

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# QuTiP for quantum objects, states, operators, and fidelity
try:
    from qutip import basis, ket2dm, tensor, fidelity, qeye
except ImportError:
    print("Please install QuTiP with: pip install qutip")
    import sys
    sys.exit(1)

###############################################################################
# 1) Define the relevant quantum states
###############################################################################
# Basis vectors |H> and |V>
H = basis(2, 0)
V = basis(2, 1)

# |Φ⁺> = 1/sqrt(2) (|HH> + |VV>)
Phi_plus = (tensor(H, H) + tensor(V, V)).unit()

# ρ = 0.7 |Φ⁺><Φ⁺| + 0.15 |HV><HV| + 0.15 |VH><VH|
rho = 0.7 * ket2dm(Phi_plus) \
    + 0.15 * ket2dm(tensor(H, V)) \
    + 0.15 * ket2dm(tensor(V, H))

# |φ> = 1/sqrt(5) |HH> + 2/sqrt(5) |VV>
phi = (1/np.sqrt(5) * tensor(H, H) + 2/np.sqrt(5) * tensor(V, V)).unit()

# ρ' = 0.7 |φ><φ| + 0.15 |HV><HV| + 0.15 |VH><VH|
rho_prime = 0.7 * ket2dm(phi) \
    + 0.15 * ket2dm(tensor(H, V)) \
    + 0.15 * ket2dm(tensor(V, H))

# Put them in a dictionary for easy iteration
all_states = {
    "Phi_plus (pure)": ket2dm(Phi_plus),
    "rho (mixed ME)": rho,
    "phi (non-max pure)": ket2dm(phi),
    "rho_prime (non-max mixed)": rho_prime
}


###############################################################################
# 2) Simulation parameters
###############################################################################
# Average entangled-photon-pair generation rate (Poisson)
ENTANGLEMENT_RATE = 15000  # pairs per second

# Detector parameters
DETECTOR_EFFICIENCY = 0.10    # 10%
DARK_COUNT_RATE = 1000        # 1000 Hz
DEAD_TIME = 4e-6              # 4 microseconds

# Coincidence window
COINCIDENCE_WINDOW = 1e-9     # 1 nanosecond

# Total run time (seconds)
TOTAL_TIME = 30

# For reproducibility (remove or change seed as needed)
np.random.seed(42)


###############################################################################
# 3) Simplified detection simulation
###############################################################################
def simulate_detection_and_timestamps(rate_source,
                                      rate_dark,
                                      eff_detector,
                                      dead_time,
                                      total_time):
    """
    Simulate detection events (arrival times) over 'total_time'.

    rate_source: Rate of "true" signal (photons or pairs).
    rate_dark: Dark-count rate (counts/s).
    eff_detector: Detector efficiency (fraction).
    dead_time: No detection can occur during this time after a detection.
    total_time: Simulation duration in seconds.

    Returns: A sorted numpy array of detection timestamps (in seconds).
    """
    # Use an event-driven approach:
    # Generate times of "true" signal with Poisson process,
    # plus times of dark counts with Poisson process,
    # combine, sort, and enforce dead time.

    # 1) Generate random times for actual photons
    #    The number of photons in total_time ~ Poisson(rate_source * total_time).
    n_source = np.random.poisson(rate_source * total_time)
    #    Each arrival time is uniform in [0, total_time], or we can treat it
    #    as an ordered Poisson process. For simplicity, we do uniform here:
    source_times = np.random.rand(n_source) * total_time

    # 2) Generate random times for dark counts
    n_dark = np.random.poisson(rate_dark * total_time)
    dark_times = np.random.rand(n_dark) * total_time

    # 3) Combine and sort
    all_times = np.concatenate((source_times, dark_times))
    all_times.sort()

    # 4) Probabilistic detection (efficiency)
    #    We do a Bernoulli trial for each event to see if it is detected.
    #    For "true" source events we also multiply by DETECTOR_EFFICIENCY;
    #    for dark counts, they "occur" with certainty but we also do the same
    #    detection pass? Many ways to handle it; here we treat them equally:
    detect_mask = np.random.rand(len(all_times)) < eff_detector
    detect_times = all_times[detect_mask]

    # 5) Enforce dead time: after a detection, ignore subsequent events
    #    that occur within 'dead_time'.
    final_times = []
    last_detection = -np.inf
    for t in detect_times:
        if t - last_detection >= dead_time:
            final_times.append(t)
            last_detection = t

    return np.array(final_times)


def count_coincidences(times_a, times_b, coincidence_window):
    """
    Count how many pairs of detection times are within 'coincidence_window'.
    A naive approach: for each detection time in 'times_a', we find any
    detection times in 'times_b' that lie within +/- coincidence_window/2
    (or just +/- window if we choose). For small arrays, a double loop works;
    for large arrays, we can do better with a two-pointer approach.

    Returns: The number of coincident pairs.
    """
    # Two-pointer approach
    i, j = 0, 0
    n_coinc = 0
    while i < len(times_a) and j < len(times_b):
        dt = times_a[i] - times_b[j]
        if abs(dt) <= coincidence_window:
            # We have a coincidence
            n_coinc += 1
            i += 1
            j += 1
        elif dt > 0:
            # times_b[j] is behind times_a[i], move b forward
            j += 1
        else:
            # times_a[i] is behind times_b[j], move a forward
            i += 1
    return n_coinc


###############################################################################
# 4) Bell measurement & CHSH S-parameter
###############################################################################
def projector(theta):
    """
    Projector onto the state:
        cos(theta)|H> + sin(theta)|V>
    Returns a rank-1 projector in QuTiP representation (2x2).
    """
    # state vector in the single-qubit space
    state = np.cos(theta) * H + np.sin(theta) * V
    return ket2dm(state.unit())


def compute_chsh_s(rho_2qubit):
    """
    Compute the CHSH S-value for the 2-qubit state 'rho_2qubit'
    using the typical measurement angles:
        A = 0°, A' = 45°,
        B = 22.5°, B' = 67.5°.

    S = E(A,B) - E(A,B') + E(A',B) + E(A',B'),
    where E(...) is the expectation of the correlation (±1 outcomes).
    """
    # Convert degrees to radians
    a = 0.0 * np.pi/180
    ap = 45.0 * np.pi/180
    b = 22.5 * np.pi/180
    bp = 67.5 * np.pi/180

    # For a single angle θ, we define projector P(θ).
    # The "observable" for spin-like correlation is:
    #     O(θ) = P(θ) - P(θ+π/2)
    # In polarization language, measuring "H/V" ~ (0°, 90°),
    # or "D/A" ~ (45°, 135°), etc.
    #
    # A more direct approach for CHSH:
    # E(a, b) = Pr(++|ab) + Pr(--|ab) - Pr(+-|ab) - Pr(-+|ab)
    #
    # We'll do a short routine to compute E(...) for given angles a, b.
    return (
        E_ab(rho_2qubit, a, b)
        - E_ab(rho_2qubit, a, bp)
        + E_ab(rho_2qubit, ap, b)
        + E_ab(rho_2qubit, ap, bp)
    )


def E_ab(rho_2qubit, theta_a, theta_b):
    """
    Compute E(a, b) = Probability(++ or --) - Probability(+- or -+),
    measuring single-qubit angles theta_a and theta_b on qubit A and B.
    """
    # Define rank-1 projectors for the + outcome:
    P_plus_A = projector(theta_a)
    P_plus_B = projector(theta_b)

    # The complementary projectors (for the - outcome) are:
    P_minus_A = (qeye(2) - P_plus_A)
    P_minus_B = (qeye(2) - P_plus_B)

    # Probability of + + is  <P_plus_A \otimes P_plus_B>
    pp = (P_plus_A.tensor(P_plus_B) * rho_2qubit).tr()
    # Probability of - - is <P_minus_A \otimes P_minus_B>
    mm = (P_minus_A.tensor(P_minus_B) * rho_2qubit).tr()
    # Probability of + - is <P_plus_A \otimes P_minus_B>
    pm = (P_plus_A.tensor(P_minus_B) * rho_2qubit).tr()
    # Probability of - + is <P_minus_A \otimes P_plus_B>
    mp = (P_minus_A.tensor(P_plus_B) * rho_2qubit).tr()

    return (pp + mm) - (pm + mp)


###############################################################################
# 5) Putting it all together
###############################################################################
def run_simulation_for_state(state_name, rho_2qubit):
    """
    1. Generate detection timestamps for Alice and Bob.
    2. Count total detection rate, coincidence rate.
    3. Compute fidelity with ideal |Φ+>.
    4. Compute Bell S parameter.
    """
    # 1) Simulate detection event times for Alice and Bob
    alice_times = simulate_detection_and_timestamps(
        rate_source=ENTANGLEMENT_RATE,    # For her half of the pairs
        rate_dark=DARK_COUNT_RATE,
        eff_detector=DETECTOR_EFFICIENCY,
        dead_time=DEAD_TIME,
        total_time=TOTAL_TIME
    )
    bob_times = simulate_detection_and_timestamps(
        rate_source=ENTANGLEMENT_RATE,    # For Bob's half
        rate_dark=DARK_COUNT_RATE,
        eff_detector=DETECTOR_EFFICIENCY,
        dead_time=DEAD_TIME,
        total_time=TOTAL_TIME
    )

    total_count_rate_alice = len(alice_times) / TOTAL_TIME
    total_count_rate_bob   = len(bob_times)   / TOTAL_TIME

    # 2) Count coincidences (very simplified),
    #    ignoring which measurement basis was used, just a raw coincidence.
    n_coinc = count_coincidences(alice_times, bob_times, COINCIDENCE_WINDOW)
    coincidence_rate = n_coinc / TOTAL_TIME

    # 3) Fidelity with |Φ+>
    #    QuTiP has a direct fidelity function:
    F = fidelity(rho_2qubit, ket2dm(Phi_plus))

    # 4) Bell S value from the CHSH scenario:
    #    We do a purely "state-based" CHSH calculation here, ignoring
    #    the detection inefficiencies, for demonstration.
    S = compute_chsh_s(rho_2qubit)

    print(f"=== Results for state: {state_name} ===")
    print(f"  Total Count Rate (Alice)  = {total_count_rate_alice:.2f} counts/s")
    print(f"  Total Count Rate (Bob)    = {total_count_rate_bob:.2f} counts/s")
    print(f"  Coincidence Rate         = {coincidence_rate:.2f} counts/s")
    print(f"  Fidelity w.r.t. |Phi+>   = {F:.3f}")
    print(f"  Bell S value (CHSH)      = {S:.3f}")
    print()


def main():
    """
    Main driver: loop over the four states and run the simulation + analytics.
    """
    for name, state in all_states.items():
        run_simulation_for_state(name, state)

    # Optional: If you want a quick demonstration or comparison plot
    # of S vs. p in a Werner state, you could do something like:
    # (CSC 791 extension: Searching for minimal S > 2 that still violates.)
    # Uncomment if desired.

    import qutip
    p_vals = np.linspace(0.0, 1.0, 21)
    s_vals = []
    for p in p_vals:
        # Werner state: rho_W = p|Phi+><Phi+| + (1-p)I/4
        rho_W = p * ket2dm(Phi_plus) + (1 - p) * (qeye(4) / 4)
        s_vals.append(compute_chsh_s(rho_W))
    
    plt.figure()
    plt.plot(p_vals, s_vals, marker='o')
    plt.xlabel("p (Werner state parameter)")
    plt.ylabel("CHSH S-value")
    plt.title("Werner State CHSH S vs. p")
    plt.show()


if __name__ == "__main__":
    main()
